In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import psycopg2

%matplotlib inline
import matplotlib.pyplot as plt

In [7]:
# Connect to database
conn = psycopg2.connect("dbname='dreamteam' user='weisihan' host='localhost' port='5432' password='weisi9527sj'")
cur = conn.cursor()
# load csgo.profiles
cur.execute('SELECT * FROM csgo.profiles')
colnames = [desc[0] for desc in cur.description]
csgo_profiles = cur.fetchall()
csgo_profiles = pd.DataFrame(csgo_profiles, columns = colnames)

# load games.profiles_roles
cur.execute('SELECT * FROM games.profiles_roles')
colnames = [desc[0] for desc in cur.description]
games_profiles_roles = cur.fetchall()
games_profiles_roles = pd.DataFrame(games_profiles_roles, columns = colnames)

# load core.profiles
cur.execute('SELECT * FROM core.profiles')
colnames = [desc[0] for desc in cur.description]
core_profiles = cur.fetchall()
core_profiles = pd.DataFrame(core_profiles, columns = colnames)

# load csgo.weapons_stats
cur.execute('SELECT * FROM csgo.weapons_stats')
colnames = [desc[0] for desc in cur.description]
csgo_weapons_stats = cur.fetchall()
csgo_weapons_stats = pd.DataFrame(csgo_weapons_stats, columns = colnames)

cur.execute('SELECT * FROM csgo.weapons')
colnames = [desc[0] for desc in cur.description]
csgo_weapons = cur.fetchall()
csgo_weapons = pd.DataFrame(csgo_weapons, columns = colnames)

cur.close()
conn.close()

## 1. Weapon Data Cleaning

In [8]:
csgo_weapons_stats = csgo_weapons_stats[csgo_weapons_stats['total_shots']!=0]
csgo_weapons_stats = csgo_weapons_stats[csgo_weapons_stats['total_hits']!=0]
csgo_weapons_stats = csgo_weapons_stats[csgo_weapons_stats['total_kills']!=0]

In [9]:
csgo_weapons_stats.shape

(654144, 5)

In [10]:
csgo_weapons_stats['accuracy'] = csgo_weapons_stats['total_hits']/csgo_weapons_stats['total_shots']
csgo_weapons_stats['K/H'] = csgo_weapons_stats['total_kills']/csgo_weapons_stats['total_hits']
csgo_weapons_stats.head()

,user_id,weapon_name,total_kills,total_hits,total_shots,accuracy,K/H
0,49773,deagle,1595,4221,23008,0.183458,0.377873
1,49773,hkp2000,1515,6061,37673,0.160884,0.249959
2,49773,glock,1391,6271,44151,0.142035,0.221815
3,49773,ak47,9931,38327,275239,0.139250,0.259112
4,49773,m4a1,3937,16831,104204,0.161520,0.233914


In [12]:
#delete outliers
csgo_weapons_stats = csgo_weapons_stats[csgo_weapons_stats['accuracy']<1]
csgo_weapons_stats = csgo_weapons_stats[csgo_weapons_stats['K/H']<1]
csgo_weapons_stats.shape

(645031, 7)

In [14]:
csgo_weapons_stats.head(100)

,user_id,weapon_name,total_kills,total_hits,total_shots,accuracy,K/H
0,49773,deagle,1595,4221,23008,0.183458,0.377873
1,49773,hkp2000,1515,6061,37673,0.160884,0.249959
2,49773,glock,1391,6271,44151,0.142035,0.221815
3,49773,ak47,9931,38327,275239,0.139250,0.259112
4,49773,m4a1,3937,16831,104204,0.161520,0.233914
5,49773,galilar,411,2089,6875,0.303855,0.196745
6,49773,awp,5479,6782,27465,0.246932,0.807874
7,49773,ump45,751,3353,18008,0.186195,0.223979
8,49773,p90,740,3484,12901,0.270057,0.212400
9,5529,hkp2000,8513,29329,156879,0.186953,0.290259


## 2. Get the top users for each role(sniper, assult, support)

In [13]:
sniper_list = ['awp', 'scar20', 'ssg08', 'g3sg1', 'hkp2000', 'deagle']
assult_list = ['ak47', 'm4a1', 'ump45', 'famas', 'aug', 'galilar']
support_list = ['sg556', 'p90', 'ump45', 'hkp2000']

In [ ]:
def split_by_roles(user_data):
    snipers = user_data[user_data.weapon_name.isin(sniper_list)]
    assults = user_data[user_data.weapon_name.isin(assult_list)]
    supports = user_data[user_data.weapon_name.isin(support_list)]
    return snipers, assults, supports

In [ ]:
def accuracy_quantiles_by_roles(snipers, assults, supports):
    roles = [snipers, assults, supports]
    user_weapons_lists = [list(set(snipers.weapon_name)), list(set(assults.weapon_name)), list(set(supports.weapon_name))]
    quantiles = [{}, {}, {}]
    
    for i in range(len(quantiles)):
        for weapon in user_weapons_lists[i]:
            temp = weapons_stats[weapons_stats.weapon_name==weapon].sort_values(by = 'accuracy').reset_index()
            position = temp[temp.accuracy==float(roles[i][roles[i].weapon_name==weapon].accuracy)].index.values/temp.shape[0]
            quantiles[i][weapon] = position[0]
            
    return quantiles